In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when

spark = SparkSession.builder.appName("CustomerOrdersJob").getOrCreate()

In [ ]:
customers_path = "/opt/spark-apps/input/customers.csv"
orders_path = "/opt/spark-apps/input/orders.json"
output_path_csv = "/opt/spark-apps/output/orders_enriched_csv"
output_path_parquet = "/opt/spark-apps/output/orders_enriched_parquet"

# ✅ Prod Mode (HDFS)
# customers_path = "hdfs://hdfs-namenode:9000/input/customers.csv"
# orders_path = "hdfs://hdfs-namenode:9000/input/orders.json"
# output_path_csv = "hdfs://hdfs-namenode:9000/output/orders_enriched_csv"
# output_path_parquet = "hdfs://hdfs-namenode:9000/output/orders_enriched_parquet"

In [14]:
df_customers = spark.read.option("header", True).csv(customers_path)
df_orders = spark.read.json(orders_path)

In [15]:
df_joined = df_orders.join(df_customers, on="customer_id", how="inner")

In [16]:
df_customers.show(), df_orders.show(), df_joined.show()

+-----------+----------+----------------+---+-------+
|customer_id|      name|           email|age|country|
+-----------+----------+----------------+---+-------+
|        101|  John Doe|john@example.com| 28|     US|
|        102|Jane Smith|jane@example.com| 34|     UK|
|        103|  Ali Khan| ali@example.com| 40|    UAE|
|        104|Rita Mehra|rita@example.com| 25|  India|
|        105| Wei Zhang| wei@example.com| 31|  China|
+-----------+----------+----------------+---+-------+

+------+-----------+--------+----------+
|amount|customer_id|order_id|    status|
+------+-----------+--------+----------+
| 250.5|        101|    5001|   shipped|
| 145.0|        102|    5002| cancelled|
|389.99|        104|    5003|processing|
| 89.99|        105|    5004|   shipped|
| 199.0|        106|    5005|  returned|
+------+-----------+--------+----------+

+-----------+------+--------+----------+----------+----------------+---+-------+
|customer_id|amount|order_id|    status|      name|           

(None, None, None)

In [17]:
df_enriched = df_joined.withColumn(
    "order_type",
    when(df_joined.amount >= 200, "High Value")
    .when(df_joined.amount >= 100, "Medium Value")
    .otherwise("Low Value")
)

In [18]:
df_enriched.show()

+-----------+------+--------+----------+----------+----------------+---+-------+------------+
|customer_id|amount|order_id|    status|      name|           email|age|country|  order_type|
+-----------+------+--------+----------+----------+----------------+---+-------+------------+
|        101| 250.5|    5001|   shipped|  John Doe|john@example.com| 28|     US|  High Value|
|        102| 145.0|    5002| cancelled|Jane Smith|jane@example.com| 34|     UK|Medium Value|
|        104|389.99|    5003|processing|Rita Mehra|rita@example.com| 25|  India|  High Value|
|        105| 89.99|    5004|   shipped| Wei Zhang| wei@example.com| 31|  China|   Low Value|
+-----------+------+--------+----------+----------+----------------+---+-------+------------+



In [19]:
df_enriched.select("order_id", "name", "amount", "order_type").show()

+--------+----------+------+------------+
|order_id|      name|amount|  order_type|
+--------+----------+------+------------+
|    5001|  John Doe| 250.5|  High Value|
|    5002|Jane Smith| 145.0|Medium Value|
|    5003|Rita Mehra|389.99|  High Value|
|    5004| Wei Zhang| 89.99|   Low Value|
+--------+----------+------+------------+



In [20]:
df_enriched_op = df_enriched.select("order_id", "name", "amount", "order_type")
df_enriched_op.write.mode("overwrite").option("header", True).csv(output_path_csv)
df_enriched_op.write.mode("overwrite").parquet(output_path_parquet)

In [21]:
df_enriched_op.write.mode("overwrite").csv(output_path_csv)